# Clean AdresMusea

### Import packages and set pandas options

In [1]:
import numpy as np
import pandas as pd
import tldextract

pd.set_option('max_row', None)
pd.set_option('max_columns', None)

### Load CSV's

In [3]:
musea_df = pd.read_csv('musea.csv')
adres_musea = pd.read_excel('AdresData Musea.xlsx', header=1)
print (len(musea_df))
musea_df.drop_duplicates(subset=['translationSetId'], inplace=True)
print (len(musea_df))

683
496


### Edit and Compare by website

In [4]:
musea_df['site_sld'] = musea_df['website']
adres_musea['site_sld'] = adres_musea['website']

adres_musea.site_sld = adres_musea['site_sld'].apply(lambda url: tldextract.extract(url).domain if type(url) == str else url)
adres_musea.site_sld = adres_musea.site_sld.str.lower()
musea_df.site_sld = musea_df['site_sld'].apply(lambda url: tldextract.extract(url).domain if type(url) == str else url)
musea_df.site_sld = musea_df.site_sld.str.lower()

q42_df1 = musea_df[['publicName', 'site_sld', 'translationSetId']]
extra_df1 = adres_musea[['site_sld', 'aantal bezoekers p/jr', 'museaal_thema']]
extra_df1 = extra_df1.drop_duplicates(subset=['site_sld'])

In [5]:
print (len(q42_df1))
q42_df1['site_sld'].isin(extra_df1['site_sld']).value_counts()

496


True     404
False     92
Name: site_sld, dtype: int64

In [6]:
site_df = q42_df1.merge(extra_df1, how="inner", left_on='site_sld', right_on='site_sld')
site_df.drop(['site_sld'], axis=1, inplace=True)
print (site_df.shape)
site_df.head()

(404, 4)


,publicName,translationSetId,aantal bezoekers p/jr,museaal_thema
0,Afrika Museum,b5023390-ced1-4210-a14c-23c349b2ee15,70000.0,volkenkunde
1,Agrarisch Museum Westerhem,1c731866-d25c-489a-b1ea-1e04a1cca554,1200.0,archeologie/historie
2,Betje Wolff Museum,42890c86-06cd-4ea5-a170-57c95dbfd26a,1200.0,archeologie/historie
3,Airborne Museum at Hartenstein,a7e42402-9eb9-4312-abf5-ae09e951e068,120000.0,"archeologie/historie, cultuurhistorie"
4,Allard Pierson,6ed658b1-4ce7-4df9-9472-4b0271d044a3,90000.0,archeologie/historie


### Edit and Compare by museum name

In [7]:
musea_df['naam'] = musea_df['publicName']
adres_musea['naam'] = adres_musea['organisatie_naam']

musea_df.naam = musea_df.naam.str.lower().str.replace('museum','')
musea_df.naam = musea_df.naam.str.replace('de','')
musea_df.naam = musea_df.naam.str.replace('het','')
musea_df.naam = musea_df.naam.str.replace('[^\w\s]','')
musea_df.naam = musea_df.naam.replace('\s+', ' ', regex=True)
musea_df.naam = musea_df.naam.str.strip()

adres_musea.naam = adres_musea.naam.str.lower().str.replace('museum','')
adres_musea.naam = adres_musea.naam.str.replace('de','')
adres_musea.naam = adres_musea.naam.str.replace('het','')
adres_musea.naam = adres_musea.naam.str.replace('[^\w\s]','')
adres_musea.naam = adres_musea.naam.replace('\s+', ' ', regex=True)
adres_musea.naam = adres_musea.naam.str.strip()



# musea_df.naam = musea_df.naam.str.lower().str.replace('^[a-zA-Z]*$','')
# musea_df.naam = musea_df.naam.str.replace('[^\w\s]','')
# musea_df.naam = musea_df.naam.str.replace('museum','').replace('\s+', ' ', regex=True)
# musea_df.naam = musea_df.naam.str.strip()

In [8]:
q42_df2 = musea_df[['publicName', 'translationSetId', 'naam']]
extra_df2 = adres_musea[['naam', 'aantal bezoekers p/jr', 'museaal_thema']]

In [9]:
# musea_df[musea_df['naam'].str.contains('sypesteyn')][['naam']]
# adres_musea[adres_musea['naam'].str.contains('sypesteyn')][['naam']]

In [10]:
print (len(musea_df[~musea_df['naam'].isin(adres_musea['naam'])][['naam']]))
# musea_df[~musea_df['naam'].isin(adres_musea['naam'])][['naam']]

158


In [11]:
naam_df = q42_df2.merge(extra_df2, how="inner", left_on='naam', right_on='naam')
naam_df.drop(['naam'], axis=1, inplace=True)
print (naam_df.shape)
naam_df.head()

(340, 4)


,publicName,translationSetId,aantal bezoekers p/jr,museaal_thema
0,Afrika Museum,b5023390-ced1-4210-a14c-23c349b2ee15,70000.0,volkenkunde
1,Agrarisch Museum Westerhem,1c731866-d25c-489a-b1ea-1e04a1cca554,1200.0,archeologie/historie
2,Allard Pierson,6ed658b1-4ce7-4df9-9472-4b0271d044a3,90000.0,archeologie/historie
3,Amsterdam Pipe Museum,b5a41500-a10f-418f-8903-6e60d0907060,5500.0,archeologie/historie
4,Anton Pieck Museum,68c00e09-7dd1-4002-a7c5-8aa69d3c589e,22655.0,beeldende kunst


### Concat two df of similar websites and museum name

In [12]:
result = pd.concat([site_df, naam_df]).drop_duplicates().reset_index(drop=True)
result.head()

,publicName,translationSetId,aantal bezoekers p/jr,museaal_thema
0,Afrika Museum,b5023390-ced1-4210-a14c-23c349b2ee15,70000.0,volkenkunde
1,Agrarisch Museum Westerhem,1c731866-d25c-489a-b1ea-1e04a1cca554,1200.0,archeologie/historie
2,Betje Wolff Museum,42890c86-06cd-4ea5-a170-57c95dbfd26a,1200.0,archeologie/historie
3,Airborne Museum at Hartenstein,a7e42402-9eb9-4312-abf5-ae09e951e068,120000.0,"archeologie/historie, cultuurhistorie"
4,Allard Pierson,6ed658b1-4ce7-4df9-9472-4b0271d044a3,90000.0,archeologie/historie


In [13]:
musea_df['translationSetId'].isin(result['translationSetId']).value_counts()

True     437
False     59
Name: translationSetId, dtype: int64

### Show the not matching musea

In [14]:
musea_df[~musea_df['publicName'].isin(result['publicName'])][['publicName']]

,publicName
9,"BAK, basis voor actuele kunst"
15,Biesbosch MuseumEiland
27,Cromhouthuis & Bijbels Museum
48,Embassy of the Free Mind
61,Geert Groote Huis
67,Museum W
70,Gorcums Museum
83,Het Nieuwe Instituut
112,Kunsthal KAdE
117,Kasteel Duivenvoorde


In [15]:
musea_df[musea_df['naam'].str.contains('rijks')][['naam']]

,naam
324,rijks boerhaave
325,rijks twenthe
326,rijks van oudhen
414,rijks amsterdam
415,rijks muirslot


In [16]:
adres_musea[adres_musea['naam'].str.contains('rijks')][['naam']]

,naam
73,rijks boerhaave
558,stg rijks muirslot
637,rijks van oudhen
725,rijksdienst voor cultureel erfgoed
726,rijks
928,rijks twenthe


# Categories / one-hot

In [29]:
one_hot_cat = result.museaal_thema.str.get_dummies(', ')
output = pd.concat([result, one_hot_cat], axis=1)
output.drop(['museaal_thema'], axis=1, inplace=True)
output.drop(['overig'], axis=1, inplace=True)
output.rename(columns={"maritiem/scheepvaart": "Naval", "natuurhistorie": "Nature", "transport/technisch": "Tech", "archeologie/historie": "History", "aantal bezoekers p/jr": "Visitors", "cultuurhistorie": "Culture", "beeldende kunst": "Visual", "volkenkunde": "Ethnology"} , inplace=True)


In [30]:
output

,publicName,translationSetId,Visitors,History,Visual,Culture,Naval,Nature,Tech,Ethnology
0,Afrika Museum,b5023390-ced1-4210-a14c-23c349b2ee15,70000.0,0,0,0,0,0,0,1
1,Agrarisch Museum Westerhem,1c731866-d25c-489a-b1ea-1e04a1cca554,1200.0,1,0,0,0,0,0,0
2,Betje Wolff Museum,42890c86-06cd-4ea5-a170-57c95dbfd26a,1200.0,1,0,0,0,0,0,0
3,Airborne Museum at Hartenstein,a7e42402-9eb9-4312-abf5-ae09e951e068,120000.0,1,0,1,0,0,0,0
4,Allard Pierson,6ed658b1-4ce7-4df9-9472-4b0271d044a3,90000.0,1,0,0,0,0,0,0
5,Anne Frank Huis,98910334-b730-446f-9b09-a7cf440699f6,1195456.0,0,0,1,0,0,0,0
6,Anton Pieck Museum,68c00e09-7dd1-4002-a7c5-8aa69d3c589e,22655.0,0,1,0,0,0,0,0
7,De Appel,cbebd2e6-4fd8-4bd8-a455-5310d4895828,18675.0,0,1,0,0,0,0,0
8,Luchtvaartmuseum Aviodrome,3caa2392-510f-4c85-a489-b83e8200ff90,200000.0,0,0,1,0,0,1,0
9,Bakkerijmuseum De Oude Bakkerij,61b12cb1-03ba-4818-b2a1-031d93ae56b1,50000.0,1,0,1,0,0,0,0


In [31]:
output.to_csv('museua_visitors.csv', index=False)
